In [ ]:
#@title connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title check the gpu before running, should be P100
import os
os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read()

'Tesla P100-PCIE-16GB\n'

In [ ]:
#@title setup detectron2
# install dependencies: 
!pip install pyyaml==5.1
!pip install torch==1.7
!pip install torchvision==0.8.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
exit(0)

     |████████████████████████████████| 276kB 13.1MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=a5cf40d944d6ab72da5e9cae4c618b4416be517fec27f06ed7dbce8155afffd9
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 776.8MB 22kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.7.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
     |████████████████████████████████| 12.7MB 8.0MB/s 
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling to

In [ ]:
#@title setup variables and imports
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random, time, ntpath
from shutil import copyfile
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog


def print_time(log):
  current_time = time.strftime("%H:%M:%S", time.localtime())
  print(current_time+": "+log)



#base path for training images
thesis_path = "/content/drive/MyDrive/thesis"
LISA_path = "/content/drive/MyDrive/thesis/LISA dataset/"
YOLO_path = "/content/drive/MyDrive/thesis/YOLO/dataset"
COCO_path = "/content/drive/MyDrive/thesis/COCO"

#@markdown model version, increment each run
def getModel(num):
  mnum = (str)(num)
  return os.path.join(COCO_path,'model'+(str)(mnum))

#make directory if dne
#os.makedirs(model_path, exist_ok=True)
trainer = None

#fill things variable for metadata
things=[]
for line in open(os.path.join(LISA_path, "categories.txt"), "r").read().split('\n'):
  things.append(line)

print_time("printing time to test")


13:23:41: printing time to test


In [ ]:
#@title register datasets
from detectron2.structures import BoxMode 
d = ["Train", "Valid", "Test"]

data = dict()
print_time("reading data")
for h in d:
  with open(os.path.join(LISA_path, "COCO_Annotations_"+h+".json"), "r") as outfile:
      data[h.lower()] = json.load(outfile)
      outfile.close()

for h in d:
  for row in data[h.lower()]:
    row["annotations"][0]["bbox_mode"] = BoxMode.XYXY_ABS

train=data["train"]
valid=data["valid"]
test=data["test"]
print_time("finished reading data")

#register custom dataset for use
from detectron2.data import DatasetCatalog, MetadataCatalog

def get_sign_dicts_premade(data):
  return data

#data - data list in standard format
#s - string representing which dataset to use
#isCOCO - if its to be registered as a coco dataset
def registerDataset(data, s):
    DatasetCatalog.register("signs_"+s, lambda : get_sign_dicts_premade(data))
    MetadataCatalog.get("signs_"+s).set(thing_classes=things)
    print_time("registered "+s+" as a general dataset")

print_time("registering datasets")
registerDataset(train, "train")
registerDataset(valid, "valid")
registerDataset(test, "test")

sign_metadata = MetadataCatalog.get("signs_train")
print_time("datasets finished registering")

13:23:46: reading data
13:23:46: finished reading data
13:23:46: registering datasets
13:23:46: registered train as a general dataset
13:23:46: registered valid as a general dataset
13:23:46: registered test as a general dataset
13:23:46: datasets finished registering


In [ ]:
#@title clear dataset catalog if error registering
DatasetCatalog.clear()
MetadataCatalog.clear()
print_time("datasets cleared")

13:23:44: datasets cleared


In [ ]:
#@title create custom trainer class
from detectron2.engine import DefaultTrainer

#custom trainer based on example given from facebook https://github.com/facebookresearch/detectron2/blob/master/tools/train_net.py
class Trainer(DefaultTrainer):
    """
    We use the "DefaultTrainer" which contains pre-defined default logic for
    standard training workflow. They may not work for you, especially if you
    are working on a new research project. In that case you can write your
    own training loop. You can use "tools/plain_train_net.py" as an example.
    """

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        """
        Create evaluator(s) for a given dataset.
        This uses the special metadata "evaluator_type" associated with each builtin dataset.
        For your own dataset, you can simply create an evaluator manually in your
        script and do not have to worry about the hacky if-else logic here.
        """
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)




In [ ]:
#@title create cfg file and evaluate
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator


def defineCFG(cfg, model, max_iter, mnum):
  cfg.merge_from_file(model_zoo.get_config_file(model))
  cfg.DATASETS.TRAIN = ("signs_train",) 
  cfg.DATASETS.TEST = ("signs_valid",)
  cfg.TEST.EVAL_PERIOD=500
  cfg.DATALOADER.NUM_WORKERS = 2
  cfg.MODEL.WEIGHTS = os.path.join(model_path, "model_final.pth") 
  cfg.SOLVER.IMS_PER_BATCH = 2
  cfg.SOLVER.BASE_LR = 0.001  # pick a good LR was 0.00025
  cfg.SOLVER.MAX_ITER = max_iter    # larger iter for large dataset
  cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(sign_metadata.get("thing_classes")) #gets the number of classes specified for the dataset
  cfg.OUTPUT_DIR = model_path

cfg = get_cfg()

model_num = 5 #@param {type:"number"}
model_path = getModel(model_num)
model = "PascalVOC-Detection/faster_rcnn_R_50_C4.yaml" #@param ["PascalVOC-Detection/faster_rcnn_R_50_C4.yaml", "COCO-Detection/faster_rcnn_R_101_C4_3x.yaml"]
max_iter = 26000 #@param {type:"number"}
defineCFG(cfg, model, max_iter, model_num)
trainer = Trainer(cfg)
trainer.resume_or_load(resume=True)

[04/19 13:23:54 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
 

In [ ]:
#@title calculate COCO ap values based on validation set, also get the inference time
trainer.test(cfg, trainer.model)

[04/19 13:24:02 d2.data.build]: Distribution of instances among all 13 categories:
|   category    | #instances   |   category   | #instances   |   category   | #instances   |
|:-------------:|:-------------|:------------:|:-------------|:------------:|:-------------|
| pedestrianC.. | 237          | signalAhead  | 203          | speedLimit35 | 117          |
| speedLimit25  | 86           |  keepRight   | 71           |  addedLane   | 58           |
|     merge     | 59           |    yield     | 69           |   laneEnds   | 56           |
|   stopAhead   | 33           | speedLimit45 | 32           | speedLimit30 | 29           |
|    school     | 33           |              |              |              |              |
|     total     | 1083         |              |              |              |              |
[04/19 13:24:02 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choi

/usr/local/lib/python3.7/dist-packages/detectron2/modeling/roi_heads/fast_rcnn.py:154: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  filter_inds = filter_mask.nonzero()


[04/19 13:24:24 d2.evaluation.evaluator]: Inference done 11/1083. 0.2257 s / img. ETA=0:09:29
[04/19 13:24:29 d2.evaluation.evaluator]: Inference done 18/1083. 0.2258 s / img. ETA=0:11:14
[04/19 13:24:35 d2.evaluation.evaluator]: Inference done 27/1083. 0.2257 s / img. ETA=0:11:10
[04/19 13:24:40 d2.evaluation.evaluator]: Inference done 35/1083. 0.2259 s / img. ETA=0:11:08
[04/19 13:24:45 d2.evaluation.evaluator]: Inference done 51/1083. 0.2261 s / img. ETA=0:09:03
[04/19 13:24:51 d2.evaluation.evaluator]: Inference done 67/1083. 0.2265 s / img. ETA=0:08:02
[04/19 13:24:56 d2.evaluation.evaluator]: Inference done 77/1083. 0.2265 s / img. ETA=0:08:01
[04/19 13:25:01 d2.evaluation.evaluator]: Inference done 91/1083. 0.2265 s / img. ETA=0:07:35
[04/19 13:25:06 d2.evaluation.evaluator]: Inference done 101/1083. 0.2264 s / img. ETA=0:07:39
[04/19 13:25:11 d2.evaluation.evaluator]: Inference done 119/1083. 0.2263 s / img. ETA=0:07:03
[04/19 13:25:16 d2.evaluation.evaluator]: Inference done 1

OrderedDict([('bbox',
              {'AP': 62.90607216671417,
               'AP-addedLane': 75.8796424614369,
               'AP-keepRight': 64.49494897064935,
               'AP-laneEnds': 55.98019302376041,
               'AP-merge': 76.59833735157466,
               'AP-pedestrianCrossing': 67.10301064080618,
               'AP-school': 67.5445285494936,
               'AP-signalAhead': 58.299059729097415,
               'AP-speedLimit25': 60.36616767623261,
               'AP-speedLimit30': 74.24993535738683,
               'AP-speedLimit35': 54.88377401218679,
               'AP-speedLimit45': 50.25633397427023,
               'AP-stopAhead': 79.19083546096066,
               'AP-yield': 32.93217095942851,
               'AP50': 80.65553947319854,
               'AP75': 75.95992274397307,
               'APl': 76.54526166902404,
               'APm': 70.28168541089194,
               'APs': 57.15895812290547})])

In [ ]:
cfg_file = open(os.path.join(model_path, "faster-rcnn.yaml"),'w')
cfg_file.write(cfg.dump())
cfg_file.close()

In [ ]:
#@title FPS calculation
cfg.MODEL.WEIGHTS = os.path.join(model_path, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25   # set a custom testing threshold
cfg.DATASETS.TEST = ("signs_valid",)
predictor = DefaultPredictor(cfg)

im =cv2.imread("/content/drive/MyDrive/thesis/LISA dataset/aiua120306-1/frameAnnotations-DataLog02142012_003_external_camera.avi_annotations/pedestrianCrossing_1333396454.avi_image4.png")

import time
times = []
for i in range(20):
    start_time = time.time()
    outputs = predictor(im)
    delta = time.time() - start_time
    times.append(delta)
mean_delta = np.array(times).mean()
fps = 1 / mean_delta
print("Average(sec):{:.2f},fps:{:.2f}".format(mean_delta, fps))

#faster r-cnn fps test with /content/drive/MyDrive/thesis/LISA dataset/
#aiua120306-1/frameAnnotations-DataLog02142012_003_external_camera.avi
#_annotations/pedestrianCrossing_1333396454.avi_image4.png


Average(sec):0.24,fps:4.10
